In [3]:
flatten_leaves = lambda d: {k: v for sd in d.values() for k, v in (flatten_leaves(sd).items() if isinstance(sd, dict) else sd.items())}


In [5]:
data = {
    "JSONLoader": {
        "required": {
            "jq_schema": None,
        },
        "optional": {
            "content_key": None,
            "is_content_key_jq_parsable": False,
            "metadata_func": None,
            "text_content": True,
            "json_lines": False,
        }
    }
}
kwargs = {**data['JSONLoader']['required'], **data['JSONLoader']['optional']}

In [6]:
kwargs

{'jq_schema': None,
 'content_key': None,
 'is_content_key_jq_parsable': False,
 'metadata_func': None,
 'text_content': True,
 'json_lines': False}

In [1]:
d = {
    1: 12,
    2: 23,
    3: "foo"
}

In [2]:
s = d[3]

del d[3]
print(f"s: {s}")
print(f"d: {d}")

s: foo
d: {1: 12, 2: 23}


In [1]:
import chromadb

In [3]:
chroma_client = chromadb.Client()

In [4]:
collection = chroma_client.create_collection(name="my_collection")

In [5]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)


/Users/shauryapednekar/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:17<00:00, 4.63MiB/s]


In [6]:
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results)


{'ids': [['id1', 'id2']], 'distances': [[1.0403728485107422, 1.2430635690689087]], 'metadatas': [[None, None]], 'embeddings': None, 'documents': [['This is a document about pineapple', 'This is a document about oranges']], 'uris': None, 'data': None}


In [4]:
from langchain_community.vectorstores import FAISS, Chroma
from langchain_huggingface import HuggingFaceEmbeddings


In [13]:
embeddings = HuggingFaceEmbeddings(**{
        "model_name": "sentence-transformers/all-mpnet-base-v2",
        "model_kwargs": {"device": "cuda"},
        "encode_kwargs": {"normalize_embeddings": False},
    })
new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)

/home/shaurya/foo/src/ingestion-pipeline/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
docs = new_db.similarity_search("Relationship between trump and farmers?")

In [15]:
docs

[Document(page_content="LIESMAN: UNDERSTOOD --\nMNUCHIN: AND WE ALSO ANNOUNCED YESTERDAY AS IT RELATES TO AGRICULTURE, WE'RE GOING TO PROTECT OUR FARMERS. THE PRESIDENT INSTRUCTED SECRETARY PERDUE TO WORK WITHTHE REST OF THE CABINET TO PROTECT OUR FARMERS FROM UNFAIR TARGETING.\nLIESMAN: ONE OTHER QUESTION OF ABIDING CONCERN TO THE MARKET HOW POLICY IS FORMED IN THE ADMINISTRATION, CAN YOU TELL US -- DID YOU KNOW ABOUT THE PRESIDENTS THREAT OF $100 BILLION OF TARIFFS BEFORE HE ANNOUNCED IT?", metadata={'language': 'english', 'url': 'http://www.cnbc.com/2018/04/06/cnbc-exclusive-cnbc-transcript-u-s-treasury-secretary-steven-mnuchin-on-cnbcs-power-lunch-today.html', 'published': '2018-04-06T23:25:00.000+03:00', 'title': 'CNBC Exclusive: CNBC Transcript: U.S. Treasury Secretary Steven Mnuchin on CNBC’s “Power Lunch” Today', 'source': '../unzipped/financial_dataset/2018_04_112b52537b67659ad3609a234388c50a/news_0013370.json'}),
 Document(page_content="DAILY INVOLVED IN THESE CONVERSATIONS W